In [20]:
import pandas as pd
import re
from collections import Counter
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
import pytz
from collections import Counter

In [21]:
# Read the list of filenames from the configuration file
with open('../file_list.txt', 'r', encoding='utf-8') as config_file:
    file_names = config_file.read().splitlines()

# Regex pattern to match the data format
pattern = r'\[(.*?)\] (.*?): (.*)'

# Initialize an empty list to store parsed data
datalist = []
stream_count = 0
# Iterate over each specified file
for file in file_names:
    full_path = f"../data/{file}"
    with open(full_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines:
            match = re.match(pattern, line)
            if match:
                date, user, message = match.groups()
                datalist.append([date, user, message,stream_count])
    stream_count = stream_count + 1

# Create a DataFrame from the parsed data
data = pd.DataFrame(datalist, columns=["date", "user", "message","stream"])


In [22]:

data['date'] = pd.to_datetime(data['date'])


In [23]:

def convert_utc_to_cet(df, date_column='date'):
    """
    Convert UTC timestamps to Central European Time (CET/CEST) with proper DST handling
    
    Parameters:
    df (pd.DataFrame): DataFrame containing the date column
    date_column (str): Name of the column containing UTC timestamps
    
    Returns:
    pd.DataFrame: DataFrame with converted timestamps
    """
    # Make a copy to avoid modifying the original
    df = df.copy()
    
    # Ensure timestamps are UTC aware
    if df[date_column].dt.tz is None:
        df[date_column] = df[date_column].dt.tz_localize('UTC')
    elif df[date_column].dt.tz != pytz.UTC:
        df[date_column] = df[date_column].dt.tz_convert('UTC')
    
    # Convert to CET/CEST (Europe/Berlin includes proper DST handling)
    df[date_column] = df[date_column].dt.tz_convert('Europe/Berlin')
    
    return df



In [24]:
data = convert_utc_to_cet(data)

In [25]:


data["user"] = data["user"].replace("Banties1g", "banties_x")
data["user"] = data["user"].replace("banties1g", "banties_x")
data["user"] = data["user"].replace("chili_poe", "chili_con_bacon")
data["user"] = data["user"].replace("chili_conbacon", "chili_con_bacon")
data["user"] = data["user"].replace("Wirelesss_", "W1r3lesss")
data["user"] = data["user"].replace("treklul", "trek44_")
data["user"] = data["user"].replace("ttrek_", "trek44_")
data["user"] = data["user"].replace("TriplesingleJ", "TripleSingleJames")
data["user"] = data["user"].replace("uuccugr", "uwu_cougar")
data["user"] = data["user"].replace("uuccugr", "uuccugr_")


In [26]:

# Get all unique usernames
unique_users = data['user'].unique()

# Create a mapping from lowercase username to all variants

user_variants = defaultdict(set)
for user in unique_users:
    user_variants[user.lower()].add(user)

# Find usernames with different capitalization
duplicate_users = {k: v for k, v in user_variants.items() if len(v) > 1}


In [27]:
# Create a mapping from all variants to the canonical (sorted first) variant
variant_map = {}
for variants in duplicate_users.values():
    sorted_variants = sorted(variants)
    canonical = sorted_variants[0]
    for v in variants:
        variant_map[v] = canonical

# Replace usernames in 'user' column
data['user'] = data['user'].apply(lambda u: variant_map.get(u, u))

In [28]:
# Original combined lists with duplicates removed and cleaned
tv7_emotes = """flirtt Suspicion zigzag YeatCat nom omeSad shomonting thinking agafat FirstTimeClanka DORVIS gasp RISING dd uncPLS domgBruh Deafge OMEGADANCEBUTFASTER kim3 Buggin speed8 .... d32 ome54 ohok minionBike Clown Explosion hackingCD JermaSoy MathTime MoneyRain PokiShare TakingNotes :0 :3 :33 :tf: !boost +1 0pixel 1DLove 3Head 3Heading 4House 4Shrug 4Weird 5Head AAAA Acknowledged ACTINUP ADHD agahi AIM AIRBALL AIWITHTHEBRAIDS Alarm ALE Alfred AlienDance AlienPls AlienPls2 AlienPls3 Aloo Alright amongE ANGRE ANOTHERONE Ant AREYOUAGIRL AREYOUAGIRLFtxQcYellingAtYou areyoufr AreYouSeriousRightNeow arnoldHalt arthur Assept AURA AwHellNah Aware AWOO AWOOGA axelF ayo bah BAND Banger BANGER banties Barack barryArrive Barry63 BantiesPaulBeef Based BASED BatChestAbove batman Batman batJAM batPls Beatles Bedge BEG BEGGING Bello BigD bieberDougie BELIEVERS BibleThump BINGO Bleh Bloons BOOBA bog BOOM BOOMIES BOINK BORGIR Borfday brbToilet Broadcaster brb Bruh BRUHMM bruv buh buhbye buhFlipExplode BUSSIN BUSSERS bye CanIHaveADollar cannySilly catAsk catBusiness CatCozy catDespair catEat catJAM catKiss catPls catSigh catSmash CatTime catTwerk CAUGHT Caught CaughtIn4K Celebrating CHADDING characterSelected CHATTERS chatting cheerleaders ChillGuy Chillin Cinema chilling clappi Clap classic CLEAN Clueless CLIPPERS CLOWNDETECTED COCKA cokeBreak COMEHERE Concerned Considering Cooked COPIUM crabPls Crunch CS2 Cuck Cuh D: damily Damn dansi dash Dave deadassFaint Delusional DemonTiming Dentge despair DespairRyan Devious DIESOFCRINGE Dime DinkDonk Dinema doggoSlava dogJAM DogLookingWickedAndCool doid dojaPls dome44 dome32 donowall donoWall doroAunt doroBleh dorobubu doroCD DoroCheer doroFiddy doroFlex doroGHOST doroHEAD doroKick doroL doroMAD doroPray doroRip doroSoy DoroTalkingAgain Dorozea doster DOUBTERS DRAIN Drake DRAMA dreamwastaken drooling drukiDnace drukiDnace2 duaKiss dudWhat EDGE EDM EDITING emo erinNya essaying ewphop eww EZ EZdodge Exerpas Explosion eyeroll fadedthanaho FARMING FeelsBadMan FeelsDankMan FeelsBlackScreen FeelsGladMan FeelsLagMan FeelsLateMan FeelsOkayMan FeelsStrongMan FeelsTiredMan FeelsWeirdMan FeelsWowMan fein FEINFEINFEINFEINFEINFEINFEINFEI FellOff fembajJAM Fiddy FiddyWtf FINALLY firewriting FirstTime FirstTimeBackseating FirstTimeChadder FirstTimeChatter FirstTimeEmoteFail FirstTimeGooner FirstTimePepega FirstTimeTest firsttimebuh FLASHBANG flightnotL Flirt Flushed fnaf footstep forsenCD forsenLaughingAtYou ForsenSingingAtYou forsenPls fortnite fr freakbob freakyfredday freddy Freedom FUNNY g32 GAGAGA gachiGASM gachiHYPER gamily GAMBA GameplayTime GAMING GatieG Gaught GENIUS GetALoadOfThisGuy gg GIGACHAD GIGACHAIR GIGACLAUS GIGAMODS GIGAMOD gigl gkeyFlip gkeyPregnantBounce gkeySMP gkeyUwu gkeywide gkeyWiding gkitten GivenUp girlBoss gkitten glorpaga glorpdetective glorp GlorpMeeting glorprave gmoney goaler goat goblin44 GODDID Gogging GoodBye Gooner gooner GoodTake GOONING GotCaughtTrolling GotEEM gothKiss gPls greetingsladies GREEDY GROOTING GRRR GULP GuitarTime GYAT HABIBI hackingCD HACKERMANS hai HAH HaltEinfachDeineFresseDuHurensohn HandsUp happi HARAM HarryStylesKiss Headbang healed HECOOKING heh HEHE HEHEHEHA Heisenberj HELLO HELP helvete Herewego hesRight heyywithrizz HEYYY hi hiii hiiii Hmm HOBBY HOLY HolyFuck homelessPOV HowDoWeTellHer HowDoWeTellHim hue HUH HUHHHHHHHHHH iAsked ICANT idiot iDrive IFISPEAK IfYouCantSeeThisEmoteUseExclamationMark7tv Ignored IGON imback IMAGINENOTHAVING7TVGETFUCKEDNON7TVUSERSIMAGINENOTHAVING7TVGETFUCKEDNON7TVUSERSIMAGINENOTHAVING7TVGET ImNotOk ImtiredBoss INTENSEGAMING islandboy ISeeYou itsover itstime Jackass jacob1 jacob2 jacob3 jacob4 jah Jammies JARVIS Jay JermaSoy jiggy job JOB Joel joever john Johnporkiscalling JokerHAHA JokerLaugh juh JumpScared JUMPSCARE JustAChillGuy JustAnotherDay JustHowItIs justinbieber KaiCenatOhiogyatwithskibiditoiletwatchingtheWrizzhappeningrightinfrontofhimwithfanumtaxtaxingthegyat KanyeStare KEKW KENOUGH KeyShaker kim3 kittyBANGER kittyBop KKalinka KKonaW KKool kratos Lamonting LastTimeChatter lava lebronArrive lebronJAM lebronTROLL LEBRONNN lemon Lemon LetsBingo LETHERCOOK LETSFUCKINGJOE LETSGO LieMeter life Life Listening LiterallyMe Lithuanian LittleTrolling LiveReaction LL LMAOFREAKY lmao Loading LOCKIN lockedin LOL Looking LookUp lore luh lurkk luton LULE LULW MAJ Madge ManchesterUnited Massive? MarblesTime Martin matSad maxwin MeRN me: MeWhenIBuyEgyptianProperty MEGALUL mee Memories merch mhm MicTime mikuPls mindloud modCheck ModAbuse MODDING Modding mods MODS Mog monakS monday monkeyListening monkeySip MONKA monkaTOS monkaW MONKE MinionHoting MoneyRain muted mutted MUGA MVP MVPOfFarallah MYHEARTILOVEDHER myIQ MYLIFE NAILS NAILSING NAHH NAHHH NAHHHH nananAYAYA NAUR NAvsEU Nerd niceguy NOCHECKMARKS NODDERS NOIDONTTHINKSO NoMaidens NOOPERS NOOOOO NOHORNY noonecares NOSHOT NOTED notListening notxqcL NOW NOWAY NOWAYING np nt nuhuh nyehehehe nyanPls nya o7 Ogre ohhh ohhhhhhhhh OHMYGAWDD ohneFinger ohno ohSHIT oj OK Okei okak OLDWORK OM om omE ome10 ome101 ome104 ome105 ome14 ome15 ome18 ome21 ome29 ome32 ome4 ome41 ome44 ome44444444 ome47 ome5 ome51 ome52 ome55 ome57 ome67 ome69 ome79 ome808 ome83 ome9 ome90 ome96 ome99 OMEGADANCE OMEGALUL OMEGALULiguess omEE omeJAM omeJudging omeOhSHIT omeScrajj omeStare OMEYES omeWiggle OMFG omgBruh ongang OneGuy ONEMORE OnMyWayToDoroMomHouse OOOO oopsie otag OuttaPocket OVERWATCH OVERWORKING OverwhelminglyWholesome owoCheer PagBounce PagChomp PagMan Panam parasocial Parasocial PARASOCIAL paris paul Paul paulNya PauseMan PAUSENEMOGU Peace PEEPEES peepoAds peepoBox peepoBelievers peepoClap peepoComfy peepoDJ peepoDoubters peepoEvil peepoFarmer peepoFat peepoGiggles peepoHappy peepoHey peepoHug peepoKiss peepoLeave peepoLegs peepoLove peepoMarch peepoPls peepoPride peepoRiot peepoSad peepoShy peepoSmile peepoStop peepoTalk pepeAgony pepeGun PepeHands pepeJAM PepeLaugh pepePoint PepePls pepeW Pepega PepegaAim PepegaChat PepegaReading PepoG Petter Pffttt Pffttt2 phew phpk pickle PianoTime Pipege pKitten pL Please pleading plink-182 plinkVibe plonk pmo Plotge PogO PogU pointless pokiFlirt pol POLICE Pondering popipopipipopipo poroPls POVbornbefore2000 ppHop ppL ppOverheat Prayge prePffttt PRIMERS PTSD pulNya PuzzleTime qq ragebait RAGEY RAHH RainTime RAMBOLMG RareParrot ratomilton RaveDance RaveTime ratio Reacting RealForsen ReallyMad RebeccaBlack Reddit RememberTheDays RibertJam RiddleMeThis RIPBOZO RIRI Rizzler RobertJam ROFL RoxyPotato RUNNING rt ryanArrive Sadding Sadge SADge SAJ SAVEME SCATTER saythatagain scawy SCHEISSE SCHIZO SCRAPETHATSHITJOHNNY SCHTOP sdd SERIOUSLY SEXO shogaNya Shits shutup Shruge silliness sisyphus Sippin Sits skip SLAY Sleepo Smile smh Smoge SmurfHey Smurfing SNACKING SNEAK SNIFFA sob SOLARFLARE songbird sotruebestie SOYSCREAM Speechless speed1 speed2 speed21 speed25 speed32 speed4 speed44 speed8 speed88 speedVibe spfLEAN:()wiltee_()tonyhawkproskater4:-:-:_FREEWAVE3-encinoman--:enteringwalmart:-wheezethelean-123 SpeedLaugh SpeedLeft SpeedR spongePls squadHips Stare Staring steve Steve SteerR StreamEnding STREAMER STREAMERSGIVINGTHEWORSTFUCKINGTAKESINEXISTENCE StoryTime Surfing SurE sus susDog Susge SUSSY Swag swagJAM ta tak TakingNotes TeamEDWARD test THATHIT ThatsJustMe ThePaulers TheVoices TheWolfInMe Thinking Thinking2 ThisChat ThisIsMinecraft TIMEOUT Tomfoolery totallylistening TriJam TriKool TriSad TRUEING TRIVSsorry ts Tuckge tuff TWEAK typeshit typhu UGH um UltraMad unibrow unemployment unmod uwu uuh VALORANT veryDoro VeryKey VeryPog VeryPogftxQcInTheShower vibePls VibePls VIDEOGAME VIEWERS vips Voices wade Waddup waga wah waiting Waiting WAHHH WAJAJA WAIT WAITWAITWAIT WakeTheFuckUpSamuraiWeHaveACityToBurn wallE waltuh walterShocked WalterVibe War WasZumPenis WATAFUCKEDUPDAY WatchingStream WAYTOODANK wdym WeAreLive WeDoNotCare WEDIDIT WEEWOO WeGood WePaid WHATAFUCKEDUPDAY WHAT WHATTT wheresmyhug Whenyourinnerwolfreleases WideAlERT WideCatGroove wideDvaAss WideHardo WidelebronJAM widemonkaGIGAftRobertDowneyJr wideprespeedlaugh WideRaveTime wideReacting wideSpeedLaugh3 widetime WidezyzzPls wig WineTime winton Wisdom woah Wokege WOT wot wrapitup WW wowie Xd xar2EDM xdd XDoubt xJAM xqc32 xqcBOZO xqcDespair xqcFuel xqcGoofy xqcL xqcSCHIZO xqcSlam xqcTake xqcTwerk xQcVeryWide YAAAY YamesBond YANITED YAPPING YeahThatsWhatIWouldaDid YEAHHH YEP YESS YIPIEE yonose Yoink YOOLOOKATTHISCATDOINITSLILDANCYDANCEINTOABREAKDANCEMOVE Yooo YOUDIED YouGotMe YouWouldntGetIt ZAMN ZhongXina zyzzBass zyzzJAM"""

# Convert to list and clean up
emote_list = [emote.strip() for emote in tv7_emotes.split() if emote.strip()]

# Remove any remaining duplicates (though the manual cleanup should have gotten most)
unique_emotes = sorted(list(set(emote_list)))

print(f"Total unique emotes: {len(unique_emotes)}")

# Create the final shortened list
final_emote_list = unique_emotes

Total unique emotes: 941


july_2025:

In [29]:
# Filter only July 2025
july_2025 = data[(data["date"].dt.year == 2025) & (data["date"].dt.month == 7)]

# Find the first message date for each user
first_messages = data.groupby("user")["date"].min().reset_index()

# Filter users whose first message was in July 2025
new_chatters = first_messages[
    (first_messages["date"].dt.year == 2025) & (first_messages["date"].dt.month == 7)
]

# Get the number of new chatters
num_new_chatters = new_chatters["user"].nunique()

print(f"Number of new chatters in July 2025: {num_new_chatters}")
stream_counts = july_2025['stream'].value_counts().reset_index()
print(f"Number of Streams in July 2025: {len(stream_counts)}")
print(f"Number of Messages in July 2025: {july_2025.shape[0]}")
print(f"Number of Users in July 2025: {july_2025['user'].nunique()}")
user_counts = july_2025['user'].value_counts().reset_index()
user_counts.sort_values('count').tail(10)

# Count number of messages per user in July 2025
user_counts = july_2025['user'].value_counts().reset_index()

# Rename columns for clarity
user_counts.columns = ['user', 'message_count']

# Get the top 3 users
top_10_users = user_counts.head(10)

print(top_10_users)

# Initialize a Counter to store emote frequencies
emote_counter = Counter()

# Go through each message and count emotes
for message in july_2025["message"]:
    words = message.split()
    for word in words:
        if word in final_emote_list:
            emote_counter[word] += 1

# Get top 3 emotes
top_5_emotes = emote_counter.most_common(5)

print("Top 5 emotes in July 2025:")
for emote, count in top_5_emotes:
    print(f"{emote}: {count} times")

# Count all words from July messages
word_counter = Counter()

for message in july_2025["message"]:
    words = message.lower().split()
    word_counter.update(words)

# Get top 3 most common words
top_3_words = word_counter.most_common(3)

print("Top 3 words in July 2025:")
for emote, count in top_3_words:
    print(f"{emote}: {count} times")

# Make a copy to avoid SettingWithCopyWarning
july_2025 = july_2025.copy()

# Round timestamps to nearest 5-minute interval
july_2025["5min"] = july_2025["date"].dt.floor("5min")

# Count messages per 5-minute interval
message_counts = july_2025.groupby("5min").size().reset_index(name="message_count")

# Get top 5 busiest 5-minute intervals
top_5_fastest = message_counts.sort_values("message_count", ascending=False).head(5)
print("Top 5 busiest 5-minute intervals in July 2025:")
print(top_5_fastest)
# Group by stream and compute message counts and time range
stream_stats = july_2025.groupby("stream").agg(
    message_count=("message", "count"),
    start_time=("date", "min"),
    end_time=("date", "max")
).reset_index()

# Compute duration in minutes
stream_stats["duration_min"] = (stream_stats["end_time"] - stream_stats["start_time"]).dt.total_seconds() / 60

# Avoid division by zero
stream_stats = stream_stats[stream_stats["duration_min"] > 0]

# Calculate messages per minute
stream_stats["messages_per_min"] = stream_stats["message_count"] / stream_stats["duration_min"]

# Get the stream with the highest messages per minute
fastest_stream = stream_stats.sort_values("messages_per_min", ascending=False).head(1)
print("Stream with the highest messages per minute:")
print(fastest_stream)

# Get all unique stream IDs in July
all_streams = set(july_2025["stream"].unique())
# Group by user and get the set of streams each user chatted in
user_streams = july_2025.groupby("user")["stream"].apply(set)

# Filter users who chatted in every stream
active_every_stream = user_streams[user_streams == all_streams]

# Get just the user names
users_in_every_stream = active_every_stream.index.tolist()
print("chatters who chatted in every stream in July 2025:")
print(users_in_every_stream)


Number of new chatters in July 2025: 4625
Number of Streams in July 2025: 26
Number of Messages in July 2025: 179825
Number of Users in July 2025: 9840
          user  message_count
0     BenXBari           8624
1     JBIN2036           7848
2      trek44_           6030
3  Georgie1471           5324
4   cr7vaibhav           5013
5   balintboss           4639
6    W1r3lesss           4484
7     rautsi__           3970
8    SchiKen44           2976
9      Typhu25           2513
Top 5 emotes in July 2025:
LOL: 4934 times
hai: 2794 times
OMEYES: 2746 times
OOOO: 2502 times
WW: 2063 times
Top 3 words in July 2025:
the: 12611 times
you: 10652 times
i: 8784 times
Top 5 busiest 5-minute intervals in July 2025:
                          5min  message_count
44   2025-07-03 18:05:00+02:00            729
45   2025-07-03 18:10:00+02:00            609
38   2025-07-03 17:35:00+02:00            602
1376 2025-07-31 15:30:00+02:00            567
37   2025-07-03 17:30:00+02:00            551
Stream with

In [30]:
# Filter only May 2024
may_2024 = data[(data["date"].dt.year == 2024) & (data["date"].dt.month == 5)]

# Find the first message date for each user
first_messages = data.groupby("user")["date"].min().reset_index()

# Filter users whose first message was in May 2024
new_chatters = first_messages[
    (first_messages["date"].dt.year == 2024) & (first_messages["date"].dt.month == 5)
]

# Get the number of new chatters
num_new_chatters = new_chatters["user"].nunique()

print(f"Number of new chatters in May 2024: {num_new_chatters}")
stream_counts = may_2024['stream'].value_counts().reset_index()
print(f"Number of Streams in May 2024: {len(stream_counts)}")
print(f"Number of Messages in May 2024: {may_2024.shape[0]}")
print(f"Number of Users in May 2024: {may_2024['user'].nunique()}")
user_counts = may_2024['user'].value_counts().reset_index()
user_counts.sort_values('count').tail(10)

# Count number of messages per user in May 2024
user_counts = may_2024['user'].value_counts().reset_index()

# Rename columns for clarity
user_counts.columns = ['user', 'message_count']

# Get the top 10 users
top_10_users = user_counts.head(10)

print(top_10_users)

# Initialize a Counter to store emote frequencies
emote_counter = Counter()

# Go through each message and count emotes
for message in may_2024["message"]:
    words = message.split()
    for word in words:
        if word in final_emote_list:
            emote_counter[word] += 1

# Get top 5 emotes
top_5_emotes = emote_counter.most_common(5)

print("Top 5 emotes in May 2024:")
for emote, count in top_5_emotes:
    print(f"{emote}: {count} times")

# Count all words from May messages
word_counter = Counter()

for message in may_2024["message"]:
    words = message.lower().split()
    word_counter.update(words)

# Get top 3 most common words
top_3_words = word_counter.most_common(3)

print("Top 3 words in May 2024:")
for emote, count in top_3_words:
    print(f"{emote}: {count} times")

# Make a copy to avoid SettingWithCopyWarning
may_2024 = may_2024.copy()

# Round timestamps to nearest 5-minute interval
may_2024["5min"] = may_2024["date"].dt.floor("5min")

# Count messages per 5-minute interval
message_counts = may_2024.groupby("5min").size().reset_index(name="message_count")

# Get top 5 busiest 5-minute intervals
top_5_fastest = message_counts.sort_values("message_count", ascending=False).head(5)
print("Top 5 busiest 5-minute intervals in May 2024:")
print(top_5_fastest)

# Group by stream and compute message counts and time range
stream_stats = may_2024.groupby("stream").agg(
    message_count=("message", "count"),
    start_time=("date", "min"),
    end_time=("date", "max")
).reset_index()

# Compute duration in minutes
stream_stats["duration_min"] = (stream_stats["end_time"] - stream_stats["start_time"]).dt.total_seconds() / 60

# Avoid division by zero
stream_stats = stream_stats[stream_stats["duration_min"] > 0]

# Calculate messages per minute
stream_stats["messages_per_min"] = stream_stats["message_count"] / stream_stats["duration_min"]

# Get the stream with the highest messages per minute
fastest_stream = stream_stats.sort_values("messages_per_min", ascending=False).head(1)
print("Stream with the highest messages per minute:")
print(fastest_stream)

# Get all unique stream IDs in May
all_streams = set(may_2024["stream"].unique())

# Group by user and get the set of streams each user chatted in
user_streams = may_2024.groupby("user")["stream"].apply(set)

# Filter users who chatted in every stream
active_every_stream = user_streams[user_streams == all_streams]

# Get just the user names
users_in_every_stream = active_every_stream.index.tolist()
print("Chatters who chatted in every stream in May 2024:")
print(users_in_every_stream)

Number of new chatters in May 2024: 2266
Number of Streams in May 2024: 29
Number of Messages in May 2024: 83053
Number of Users in May 2024: 2266
             user  message_count
0    Martin_Gales           7506
1       banties_x           3670
2            oJov           3302
3       1206paul_           2506
4  StreamElements           2403
5           roxa0           2309
6           LX212           2226
7     IvanOnMyOwn           2089
8         klimzaa           1990
9         Risc__V           1527
Top 5 emotes in May 2024:
mhm: 1967 times
omE: 1368 times
o7: 625 times
OMEGALUL: 502 times
hi: 474 times
Top 3 words in May 2024:
the: 11231 times
you: 7300 times
i: 7155 times
Top 5 busiest 5-minute intervals in May 2024:
                          5min  message_count
1494 2024-05-27 23:55:00+02:00            339
1149 2024-05-22 15:45:00+02:00            292
1495 2024-05-28 00:00:00+02:00            225
1558 2024-05-28 21:55:00+02:00            186
1643 2024-05-29 19:55:00+02:00      

In [31]:
# Filter only May 2024
may_2024 = data[(data["date"].dt.year == 2025) & (data["date"].dt.month == 9)]

# Find the first message date for each user
first_messages = data.groupby("user")["date"].min().reset_index()

# Filter users whose first message was in May 2024
new_chatters = first_messages[
    (first_messages["date"].dt.year == 2025) & (first_messages["date"].dt.month == 9)
]

# Get the number of new chatters
num_new_chatters = new_chatters["user"].nunique()

print(f"Number of new chatters in Last 2024: {num_new_chatters}")
stream_counts = may_2024['stream'].value_counts().reset_index()
print(f"Number of Streams in Last 2024: {len(stream_counts)}")
print(f"Number of Messages in Last 2024: {may_2024.shape[0]}")
print(f"Number of Users in Last 2024: {may_2024['user'].nunique()}")
user_counts = may_2024['user'].value_counts().reset_index()
user_counts.sort_values('count').tail(10)

# Count number of messages per user in May 2024
user_counts = may_2024['user'].value_counts().reset_index()

# Rename columns for clarity
user_counts.columns = ['user', 'message_count']

# Get the top 10 users
top_10_users = user_counts.head(10)

print(top_10_users)

# Initialize a Counter to store emote frequencies
emote_counter = Counter()

# Go through each message and count emotes
for message in may_2024["message"]:
    words = message.split()
    for word in words:
        if word in final_emote_list:
            emote_counter[word] += 1

# Get top 5 emotes
top_5_emotes = emote_counter.most_common(10)

print("Top 10 emotes in Last 2024:")
for emote, count in top_5_emotes:
    print(f"{emote}: {count} times")

# Count all words from May messages
word_counter = Counter()

for message in may_2024["message"]:
    words = message.lower().split()
    word_counter.update(words)

# Get top 3 most common words
top_3_words = word_counter.most_common(3)

print("Top 3 words in Last 2024:")
for emote, count in top_3_words:
    print(f"{emote}: {count} times")

# Make a copy to avoid SettingWithCopyWarning
may_2024 = may_2024.copy()

# Round timestamps to nearest 5-minute interval
may_2024["5min"] = may_2024["date"].dt.floor("5min")

# Count messages per 5-minute interval
message_counts = may_2024.groupby("5min").size().reset_index(name="message_count")

# Get top 5 busiest 5-minute intervals
top_5_fastest = message_counts.sort_values("message_count", ascending=False).head(5)
print("Top 5 busiest 5-minute intervals in Last 2024:")
print(top_5_fastest)

# Group by stream and compute message counts and time range
stream_stats = may_2024.groupby("stream").agg(
    message_count=("message", "count"),
    start_time=("date", "min"),
    end_time=("date", "max")
).reset_index()

# Compute duration in minutes
stream_stats["duration_min"] = (stream_stats["end_time"] - stream_stats["start_time"]).dt.total_seconds() / 60

# Avoid division by zero
stream_stats = stream_stats[stream_stats["duration_min"] > 0]

# Calculate messages per minute
stream_stats["messages_per_min"] = stream_stats["message_count"] / stream_stats["duration_min"]

# Get the stream with the highest messages per minute
fastest_stream = stream_stats.sort_values("messages_per_min", ascending=False).head(1)
print("Stream with the highest messages per minute:")
print(fastest_stream)

# Get all unique stream IDs in May
all_streams = set(may_2024["stream"].unique())

# Group by user and get the set of streams each user chatted in
user_streams = may_2024.groupby("user")["stream"].apply(set)

# Filter users who chatted in every stream
active_every_stream = user_streams[user_streams == all_streams]

# Get just the user names
users_in_every_stream = active_every_stream.index.tolist()
print("Chatters who chatted in every stream in Last 2024:")
print(users_in_every_stream)

Number of new chatters in Last 2024: 2512
Number of Streams in Last 2024: 14
Number of Messages in Last 2024: 97725
Number of Users in Last 2024: 6460
              user  message_count
0         BenXBari           8632
1         JBIN2036           4547
2         HALP____           3013
3          Odah_02           2880
4       balintboss           2604
5  nishad_more1311           2303
6       cr7vaibhav           2267
7   StreamElements           1908
8         KrieSeaX           1847
9  InverseEntropy_           1541
Top 10 emotes in Last 2024:
LOL: 1987 times
WW: 1854 times
sob: 1768 times
hi: 1165 times
OOOO: 1065 times
mhm: 1012 times
OMEYES: 997 times
om: 994 times
ome51: 986 times
bye: 852 times
Top 3 words in Last 2024:
the: 8497 times
to: 8170 times
you: 7505 times
Top 5 busiest 5-minute intervals in Last 2024:
                         5min  message_count
271 2025-09-07 15:30:00+02:00            457
137 2025-09-03 17:15:00+02:00            403
139 2025-09-03 17:25:00+02:00    

In [32]:
# Filter July & August 2025
july_data = data[(data["date"].dt.year == 2025) & (data["date"].dt.month == 8)]
august_data = data[(data["date"].dt.year == 2025) & (data["date"].dt.month == 9)]

# --- Helper: count emotes (total, users, per stream) ---
def emote_stats(df, final_emote_list):
    counts = defaultdict(int)               # total occurrences
    users = defaultdict(set)                # unique users
    stream_counts = defaultdict(set)        # unique streams where used
    
    for _, row in df.iterrows():
        msg, user, stream = row["message"], row["user"], row["stream"]
        words = msg.split()  # split by whitespace
        for emote in final_emote_list:
            if emote in words:  # now it only counts exact whole word matches
                counts[emote] += 1
                users[emote].add(user)
                stream_counts[emote].add(stream)

    
    stats = pd.DataFrame({
        "emote": list(counts.keys()),
        "count": list(counts.values()),
        "unique_users": [len(users[e]) for e in counts.keys()],
        "streams_used": [len(stream_counts[e]) for e in counts.keys()],
    })
    return stats

# Get stats for both months
july_stats = emote_stats(july_data, final_emote_list)
august_stats = emote_stats(august_data, final_emote_list)

# --- Merge to compute relative growth ---
merged = pd.merge(august_stats, july_stats, on="emote", how="left", suffixes=("_aug", "_jul"))
merged = merged.fillna(0)  # emotes not in July become 0

# Relative growth (compared to July counts)
merged["growth"] = merged["count_aug"] - merged["count_jul"]

# Frequency per stream (normalize)
merged["freq_per_stream"] = merged["count_aug"] / merged["streams_used_aug"].replace(0, 1)


# Sort by different "trending" definitions:
top_growth = merged.sort_values("growth", ascending=False)
top_unique_users = merged.sort_values("unique_users_aug", ascending=False)
top_freq_per_stream = merged.sort_values("freq_per_stream", ascending=False)

print("📈 Trending by growth vs July:")
print(top_growth[["emote", "count_aug", "count_jul", "growth"]].head(10))

print("\n👥 Trending by unique users in August:")
print(top_unique_users[["emote", "count_aug", "unique_users_aug"]].head(10))

print("\n📊 Trending by frequency per stream:")
print(top_freq_per_stream[["emote", "count_aug", "streams_used_aug", "freq_per_stream"]].head(10))

📈 Trending by growth vs July:
            emote  count_aug  count_jul  growth
36       thinking        609      225.0   384.0
761     Suspicion        140        0.0   140.0
505        zigzag        123        0.0   123.0
156         Alarm        196       98.0    98.0
673       ISeeYou         66        0.0    66.0
420  WideRaveTime         53       13.0    40.0
324          GULP        206      171.0    35.0
416  sotruebestie         43       10.0    33.0
269    JumpScared         58       26.0    32.0
240       Devious         77       47.0    30.0

👥 Trending by unique users in August:
      emote  count_aug  unique_users_aug
6        hi       1122               398
30      LOL       1978               253
39       WW       1760               187
323     bye        611               172
44       gg        655               172
437  !boost        558               168
28     OOOO       1054               149
62       o7        495               138
63      mhm       1007            

In [33]:
# --- 1. Count messages per user ---
july_counts = july_data.groupby("user")["message"].count().reset_index(name="count_jul")
august_counts = august_data.groupby("user")["message"].count().reset_index(name="count_aug")

# Merge July + August
user_trends = pd.merge(august_counts, july_counts, on="user", how="left").fillna(0)

# Growth in messages
user_trends["growth"] = user_trends["count_aug"] - user_trends["count_jul"]

# --- 2. Unique emotes used per user (August only) ---
def count_unique_emotes(msgs, final_emote_list):
    used = set()
    for m in msgs:
        words = m.split()
        for em in final_emote_list:
            if em in words:
                used.add(em)
    return len(used)

unique_emotes = august_data.groupby("user")["message"].apply(
    lambda msgs: count_unique_emotes(msgs, final_emote_list)
).reset_index(name="unique_emotes_used")

user_trends = user_trends.merge(unique_emotes, on="user", how="left")

# --- 3. Streams participated in (August only) ---
streams_per_user = august_data.groupby("user")["stream"].nunique().reset_index(name="streams_active")
user_trends = user_trends.merge(streams_per_user, on="user", how="left")

# --- Results ---
top_growth_users = user_trends.sort_values("growth", ascending=False)
top_emote_variety = user_trends.sort_values("unique_emotes_used", ascending=False)
top_consistency = user_trends.sort_values("streams_active", ascending=False)

print("📈 Trending Users by Growth vs July:")
print(top_growth_users[["user", "count_aug", "count_jul", "growth"]].head(10))

print("\n🎨 Trending Users by Unique Emotes Used:")
print(top_emote_variety[["user", "unique_emotes_used", "count_aug"]].head(10))

print("\n🟢 Trending Users by Streams Active In:")
print(top_consistency[["user", "streams_active", "count_aug"]].head(10))

📈 Trending Users by Growth vs July:
                 user  count_aug  count_jul  growth
330          BenXBari       8632     4237.0  4395.0
4730  nishad_more1311       2303      524.0  1779.0
5944           trek_x        951      103.0   848.0
775          HALP____       3013     2376.0   637.0
853      Iiidxnm12iii        738      190.0   548.0
5036      polimpompis        571      140.0   431.0
1528          SHKADEL        384        0.0   384.0
4203         loganxdr        384        4.0   380.0
6458           알래스카해달        469      106.0   363.0
47           1SKELTON        653      290.0   363.0

🎨 Trending Users by Unique Emotes Used:
               user  unique_emotes_used  count_aug
900        JBIN2036                 393       4547
1346        Odah_02                 272       2880
330        BenXBari                 266       8632
5187       rautsi__                 233       1424
2691     cr7vaibhav                 230       2267
775        HALP____                 225      